# Защита данных клиентов

Входные данные - данные клиентов страховой компании. 

Задача - разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосновать корректность его работы. При преобразовании качество моделей машинного обучения не должно ухудшиться. Подбирать наилучшую модель не требуется.

# Оглавление

* [1. Загрузка данных](#1.-Загрузка-данных)
    * [Выводы (шаг 1)](#Выводы-(шаг-1))
* [2. Умножение матриц](#2.-Умножение-матриц)
* [3. Алгоритм преобразования](#3.-Алгоритм-преобразования)
* [4. Проверка алгоритма](#4.-Проверка-алгоритма)
    * [Без преобразования](#Без-преобразования)
    * [C преобразованием](#C-преобразованием)
    * [Выводы (шаг 4)](#Выводы-(шаг-4))

# 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [5]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [6]:
# Columns names translation
data.columns = ['gender', 'age', 'salary', 'family_members', 'insurance_payments']
data.columns

Index(['gender', 'age', 'salary', 'family_members', 'insurance_payments'], dtype='object')

In [7]:
# Check for float values
all(obj % 1 == 0 for obj in data['salary'])

False

In [8]:
# Visual float values analysis
[obj for obj in data['salary'] if obj % 1 != 0]

[32700.000000000004,
 64900.00000000001,
 32700.000000000004,
 32700.000000000004,
 32700.000000000004,
 65099.99999999999,
 32200.000000000004,
 32200.000000000004,
 32200.000000000004,
 64099.99999999999,
 32700.000000000004,
 32200.000000000004,
 32700.000000000004,
 32700.000000000004,
 32200.000000000004,
 32700.000000000004,
 32200.000000000004,
 32200.000000000004,
 32200.000000000004,
 32700.000000000004,
 32700.000000000004,
 32200.000000000004,
 32700.000000000004,
 32700.000000000004,
 32200.000000000004,
 64400.00000000001,
 32200.000000000004,
 32200.000000000004,
 32700.000000000004,
 65099.99999999999,
 32700.000000000004,
 32700.000000000004,
 32700.000000000004,
 32700.000000000004,
 32200.000000000004,
 32200.000000000004,
 32700.000000000004,
 32200.000000000004,
 32700.000000000004,
 32700.000000000004]

In [9]:
# Casting to an integer type
data['salary'] = data['salary'].astype('int')
data['gender'] = data['gender'].astype('int8')
data['age'] = data['age'].astype('int8')
data['family_members'] = data['family_members'].astype('int8')
data['insurance_payments'] = data['insurance_payments'].astype('int8')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
gender                5000 non-null int8
age                   5000 non-null int8
salary                5000 non-null int64
family_members        5000 non-null int8
insurance_payments    5000 non-null int8
dtypes: int64(1), int8(4)
memory usage: 58.7 KB


In [10]:
data.head()

,gender,age,salary,family_members,insurance_payments
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


## Выводы (шаг 1)

Файл с данными состоит из 5 столбцов (3 с плавающей точкой и 2 с целыми числами) и 5000 записей. В столбце "Страховые выплаты" содержатся целевые признаки.

Названия признаков, для удобства, были переведены на английский язык следующим образом:
- Пол -> gender
- Возраст -> age
- Зарплата -> salary
- Члены семьи -> family_members
- Страховые выплаты -> insurance_payments

В данных отсутствуют пропуски. Тип столбцов age и salary был заменен на целочисленный с целью экономии памяти ввиду того, что в столбцах отсутствуют дробные значения. Также для столбцов, где значения не могут превышать 127, тип данных был заменен на int8 с целью экономии памяти.

Исходя из максимальных и минимальных значений столбцов можно сделать вывод, что в данных отсутствуют аномалии.

# 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Не изменится

**Обоснование:** Матрица $X$ имеет размерность $(m*n)$. Матрица $P$, т.к. она обратима и на нее можно умножить матрицу $X$, имеет размерность $(n*n)$. При умножении матрицы $X$ на матрицу $P$ матрица $XP$ будет иметь размерность $(m*n)$, т.е. размерность матрицы признаков после преобразования не изменится. 

Уравнение предсказания можно записать следующим образом:

$$
a = X((X^T X)^{-1} X^T y)
$$

Умножим матрицу $X$ на матрицу $P$:

$$
a_{t} = XP(((XP)^T XP)^{-1} (XP)^T y)
$$


Раскроем полученное выражения с помощью следующих свойств матриц:
- $(AB)^T = B^T A^T$
- $(AB)^{-1} = B^{-1} A^{-1}$
- $AA^{-1} = A^{-1}A = E$
- $AE = EA = A$

$$
a_{t} = XP(((XP)^T XP)^{-1} (XP)^T y) = XP (P^T X^T XP)^{-1} P^T X^T y = XP (XP)^{-1} (P^T X^T)^{-1} P^T X^T y = XP P^{-1} X^{-1} (X^T)^{-1} (P^T)^{-1} P^T X^T y = X E X^{-1} (X^T)^{-1} E X^T y = X X^{-1} (X^T)^{-1} X^T y = X (X^T X)^{-1} X^T y = a
$$

В данном случае, учитывается, что в матрице Х умножение на матрицу P происходит для всеx столбцов, кроме нулевого: из приведенного доказательства видно, что содержание матрицы X не влияет на обоснование.

# 3. Алгоритм преобразования

**Алгоритм**

Умножить матрицу признаков X на случайную обратимую матрицу P размера ($n*n$), где n - количество признаков в датасете.

**Обоснование**

При умножении на такую матрицу качество линейной регресcии, как уже было доказано, не изменится, а восстановление возможно путем умножения полученных признаков на обратную матрицу $P^{-1}$.

$$
X = XPP^{-1} = XE = X
$$

По свойству $PP^{-1} = E$.


# 4. Проверка алгоритма

## Без преобразования

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [12]:
# Splitting data into features and target
X = data.drop(columns='insurance_payments')
y = data['insurance_payments']

In [13]:
# Splitting data into train and test samples
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [14]:
# Model fitting
model = LinearRegression(n_jobs=-1).fit(X_train, y_train)

In [15]:
# Model test with R2 metric
model.score(X_test, y_test)

0.42547785357547596

## C преобразованием

In [16]:
# Data before transfromation
X.head()

,gender,age,salary,family_members
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


In [17]:
# Check for random matrix reversibility 

reversible_matrix = False

n = X.shape[1]

while reversible_matrix != True:
    P = np.random.rand(n, n)
    
    try:
        if ((np.round_(P @ np.linalg.inv(P), 10) == np.eye(n)).all()):
            reversible_matrix = True
    except:
        continue

In [18]:
# Data transformation
X_t = data.drop(columns='insurance_payments') @ P
X_t.head()

,0,1,2,3
0,24964.192046,8204.924069,18444.760495,19427.609842
1,19134.956985,6294.742259,14135.669729,14895.046826
2,10576.632861,3480.523714,7812.498146,8234.235948
3,20979.446657,6889.987359,15503.487354,16322.325467
4,13140.549060,4321.439815,9707.518911,10228.273412


In [19]:
# Splitting transformed data into train and test samples
X_train_t, X_test_t, y_train, y_test = train_test_split(X_t, y, random_state=42)

In [20]:
# Model fitting
model_t = LinearRegression(n_jobs=-1).fit(X_train_t, y_train)

In [21]:
# Model test with R2 metric
model_t.score(X_test_t, y_test)

0.42547785357402196

In [22]:
print(f'Difference between models tests results - {abs(model_t.score(X_test_t, y_test) - model_t.score(X_test_t, y_test))}')

Difference between models tests results - 0.0


## Выводы (шаг 4)

Исходя из проведенного эксперимента видно, что качество модели до и после преобразования не отличается.

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования